In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import random
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
ment1 = pd.read_csv('Shodhganga_mentorship.csv', sep = ",")

In [4]:
def dism_problem2(uid, ment):
    uniq_adv_inst= list(pd.unique(ment[ment['advId_1']==uid]['instituteId']))
    if len(uniq_adv_inst)==1:
        return
    else:
        inst_count = ment[ment['advId_1']==uid].groupby('instituteId')['advId_1'].agg('count').reset_index()
        inst1 = inst_count[inst_count['advId_1']==max(inst_count['advId_1'])]['instituteId'].values
        for inst in uniq_adv_inst:
            if inst == inst1[0]:
                continue
            else:
                ment.loc[((ment['advId_1']==uid) & (ment['instituteId'] == inst)), 'advId_1'] = str(uid)+"_"+inst
                as_stud_inst = pd.unique(ment[ment['studId_1']==uid]['instituteId'])
                if inst in as_stud_inst:
                    ment.loc[((ment['studId_1']==uid) & (ment['instituteId']==inst)),'studId_1'] = str(uid)+"_"+inst
                else:
                    continue
        if not ment[(ment['studId_1']==uid)].empty:
            cond = ((ment['advId_1'].str.contains("^"+str(uid)+"_", na=False)) | (ment['advId_1']==uid))
            commom_dept = set(ment[cond]['DepartmentId']).intersection(ment[ment['studId_1']==uid]['DepartmentId'].values)
            print(commom_dept)
            cond2=ment['DepartmentId'].isin(commom_dept)
            inst2 = ment[(cond) & (cond2)]['instituteId'].values
            #cond1=((cond4) & (cond2))
            for i in inst2: 
                if not ment[((ment['studId_1']==uid) & (ment['instituteId']==i))].empty:
                    ment.loc[((ment['studId_1']==uid) & (ment['instituteId']==i)),'studId_1']=str(uid)+"_"+i
    return

In [5]:
def dism_problem3(uid, ment):
    uniq_adv_inst= list(pd.unique(ment[ment['advId_1']==uid]['instituteId']))
    if len(uniq_adv_inst)==1:
        return "single"
    else:
        for inst in uniq_adv_inst:
            ment.loc[((ment['advId_1']==uid) & (ment['instituteId'] == inst)), 'advId_1'] = str(uid)+"_"+inst
            as_stud_inst = pd.unique(ment[ment['studId_1']==uid]['instituteId'])
            if inst in as_stud_inst:
                ment.loc[((ment['studId_1']==uid) & (ment['instituteId']==inst)),'studId_1'] = str(uid)+"_"+inst
            else:
                continue
        if not ment[(ment['studId_1']==uid)].empty:
            cond = (ment['advId_1'].str.contains("^"+str(uid)+"_", na=False))
            commom_dept = set(ment[cond]['DepartmentId']).intersection(ment[ment['studId_1']==uid]['DepartmentId'].values)
            for dept in commom_dept:
                inst_adv = pd.unique(ment[(cond) & (ment['DepartmentId']==dept)]['instituteId'])
                ment.loc[((ment['studId_1']==uid) & (ment['DepartmentId']==dept)),'studId_1']=str(uid)+"_"+inst_adv[0]
    return "final"

In [6]:
#dism_problem2(ment1)

In [7]:
# uniq_adv_id = pd.unique(ment['advId_1'])
# for uid in tqdm(uniq_adv_id, total=len(uniq_adv_id)):
#     dism_problem3(uid, ment1)

In [8]:
uniq_adv_id = pd.Series(pd.unique(ment1['advId_1'])).copy()

In [9]:
#uniq_adv_id[499:550]

In [10]:
#uniq_adv_id

In [11]:
tmp=uniq_adv_id.progress_apply(dism_problem3,args=(ment1,))

100%|██████████| 68347/68347 [1:27:55<00:00, 12.95it/s]  


In [12]:
tmp[tmp=="single"].shape

(62790,)

In [13]:
ment1.to_csv("mod1_shodhganga_mentorship.csv", index=False)

In [ ]:
#ment2=ment1[(ment1["advId_1"]==393) |(ment1["studId_1"]==393)].copy()

In [ ]:
#dism_problem3(393, ment2)

In [ ]:
#dism_problem2(393, ment2)

In [ ]:
#ment2[ment2["studId_1"].str.contains("^393_", na=False)]

In [ ]:
#ment2[(ment2["studId_1"]==393)]

In [ ]:
#ment2[(ment2["advId_1"]=="393_I333")]

In [ ]:
#ment2[["dc.publisher.institution[]","instituteId"]].values